In [ ]:
from models.codec.kmeans.kmeans_model import KMeans, KMeansEMA
from transformers import Wav2Vec2BertModel
from transformers import SeamlessM4TFeatureExtractor
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import librosa
from IPython.display import Audio
import matplotlib.pyplot as plt

In [ ]:
kmeans_model = KMeansEMA(1024, 1024)
model_path = "/blob/v-yuancwang/codec_ckpt/semantic_kmeans/semantic_kmeans_ema_debug_libritts/checkpoint/epoch-0010_step-0002000_loss-679.313721/pytorch_model.bin"
kmeans_model.load_state_dict(torch.load(model_path))
kmeans_model.eval()
kmeans_model.cuda()

In [ ]:
wav_path_0 = "/home/t-zeqianju/yuancwang/temp_test_dataset/libritts-train-clean-100/train-clean-100/8468/294887/8468_294887_000017_000002.wav"
wav_path_1 = "/home/t-zeqianju/yuancwang/temp_test_dataset/libritts-train-clean-100/train-clean-100/5022/29405/5022_29405_000025_000003.wav"
wav_path_2 = "/home/t-zeqianju/yuancwang/temp_test_dataset/libritts-train-clean-100/train-clean-100/5022/29411/5022_29411_000051_000002.wav"
wav_path_4 = "/home/t-zeqianju/yuancwang/temp_test_dataset/libritts-train-clean-100/train-clean-100/1502/122619/1502_122619_000052_000002.wav"
# concat 4 wav
wav_0, sr = librosa.load(wav_path_0, sr=16000)
wav_1, sr = librosa.load(wav_path_1, sr=16000)
wav_2, sr = librosa.load(wav_path_2, sr=16000)
wav_4, sr = librosa.load(wav_path_4, sr=16000)
wav = np.concatenate([wav_0, wav_1, wav_2, wav_4])
Audio(wav, rate=16000)

In [ ]:
len(wav) // 320

In [ ]:
processor = SeamlessM4TFeatureExtractor.from_pretrained("facebook/w2v-bert-2.0")
semantic_model = Wav2Vec2BertModel.from_pretrained("facebook/w2v-bert-2.0")
semantic_model.eval()
semantic_model = semantic_model.cuda()
layer_idx = 15
if layer_idx == 23:
    output_idx = 0
else:
    output_idx = layer_idx + 2
stat_mean_var = torch.load("/home/t-zeqianju/yuancwang/mls_wav2vec2bert_stats.pt")
semantic_mean = stat_mean_var["mean"]
semantic_std = torch.sqrt(stat_mean_var["var"])
semantic_mean = semantic_mean.to("cuda")
semantic_std = semantic_std.to("cuda")
print("semantic mean: ", semantic_mean, "semantic std: ", semantic_std)

In [ ]:
speech_data = wav
inputs = processor(speech_data, sampll_rate=16000)
input_features = inputs["input_features"]
print(input_features.shape)
attention_mask = inputs["attention_mask"]
print(attention_mask.shape)

In [ ]:
# plot input features
plt.figure(figsize=(10, 4))
plt.imshow(input_features[0].T, aspect="auto", origin="lower")
plt.colorbar()
plt.title("Input Features")
plt.show()

In [ ]:
input_features = torch.tensor(input_features).cuda()
attention_mask = torch.tensor(attention_mask).cuda()

In [ ]:
with torch.no_grad():
    vq_emb = semantic_model(
        input_features=input_features,
        attention_mask=attention_mask,
        output_hidden_states=True,
    )
    feat = vq_emb.hidden_states[output_idx]  # B x T x C
    feat = (feat - semantic_mean.to(feat)) / semantic_std.to(feat)

In [ ]:
vq_id, _ = kmeans_model.quantize(feat)
vq_id = vq_id.cpu().numpy()[0]
print(vq_id)

In [ ]:
unique, counts = np.unique(vq_id, return_counts=True)
idx = np.argsort(counts)[::-1]
unique = unique[idx]
counts = counts[idx]
print(unique)

In [ ]:
idx = np.where(vq_id == 411)[0]
print(idx)

In [ ]:
Audio(wav[(idx[0]-5) * 320: (idx[0]+5) * 320], rate=16000)

In [ ]:
Audio(wav[(idx[1]-5) * 320: (idx[1]+5) * 320], rate=16000)

In [ ]:
Audio(wav[(idx[2]-5) * 320: (idx[2]+5) * 320], rate=16000)

In [ ]:
Audio(wav[(idx[3]-5) * 320: (idx[3]+5) * 320], rate=16000)

In [ ]:
Audio(wav[(idx[4]-5) * 320: (idx[4]+5) * 320], rate=16000)